In [1]:
import pandas as pd

base = pd.read_csv('dataset.csv')

In [2]:
import numpy as np

previsores = base.iloc[:1400000,0:20].values
previsores = np.reshape(previsores, (2000, 700, 20))
print(previsores.shape)

classes = base.iloc[:1400000, 28:31].values
classes = np.reshape(classes, (2000, 700, 3))
print(classes.shape)

(2000, 700, 20)
(2000, 700, 3)


In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, CuDNNGRU, Bidirectional, Masking

c:\users\gabriel\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\gabriel\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\gabriel\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\gabr

In [4]:
def criarRede():
    model = Sequential()
  
    #model.add(Masking(mask_value = 0, input_shape = (700, 20)))
  
    model.add(Bidirectional(CuDNNGRU(100, return_sequences = True), input_shape = (700, 20)))
    model.add(Dropout(0.3))
    
    model.add(Bidirectional(CuDNNGRU(100, return_sequences = True)))
    model.add(Dropout(0.3))

    model.add(Bidirectional(CuDNNGRU(100, return_sequences = True)))
    model.add(Dropout(0.3))
    
    model.add(Bidirectional(CuDNNGRU(100, return_sequences = True)))
    model.add(Dropout(0.3))
  
    model.add(Dense(3, activation = 'softmax'))
  
    model.compile(optimizer = 'adam', metrics = ['acc'], loss='categorical_crossentropy')
  
    return model

In [5]:
accu = []
precisions = []
recalls = []
f1 = []

In [6]:
from sklearn.metrics import balanced_accuracy_score, classification_report, precision_score, recall_score, f1_score, confusion_matrix

def train_and_evaluate_model(model, x_train, y_train, x_test, y_test):
    model.fit(x_train, y_train, epochs = 100, verbose = 1, batch_size = 32)
  
    predicted = model.predict(x_test)
  
    y_teste = []
    predict = []
  
    predicted = np.reshape(predicted, (predicted.shape[0] * predicted.shape[1], 3))
    y_test = np.reshape(y_test, (y_test.shape[0] * y_test.shape[1], 3))
    x_test = np.reshape(x_test, (x_test.shape[0] * x_test.shape[1], 20))

    for i in range(len(x_test)):
        cont = 0
        for j in range(len(x_test[i])):
            cont += x_test[i][j]
        if cont != 0:
            y_teste.append(y_test[i])
            predict.append(predicted[i])
    
    y_teste = np.asarray(y_teste)
    predict = np.asarray(predict)

    predicted = predict
    y_test = y_teste

    predicted = np.argmax(predicted, axis=1)
    y_test = np.argmax(y_test, axis=1)

    accu.append(balanced_accuracy_score(y_test, predicted)) 
    precisions.append(precision_score(y_test, predicted, average = 'weighted'))
    recalls.append(recall_score(y_test, predicted, average = 'weighted'))
    f1.append(f1_score(y_test, predicted, average = 'weighted'))
    
    print(classification_report(y_test, predicted))
    print('Acurácia')
    print(balanced_accuracy_score(y_test, predicted))
    print('Precisao')
    print(precision_score(y_test, predicted, average = 'weighted'))
    print('Recall')
    print(recall_score(y_test, predicted, average = 'weighted'))
    print('F1')
    print(f1_score(y_test, predicted, average = 'weighted'))
    print(confusion_matrix(y_test, predicted))

In [7]:
from sklearn.model_selection import KFold

kf = KFold(n_splits = 5, shuffle = True)

for train_index, test_index in kf.split(previsores):
    model = None
    model = criarRede()
    print("TRAIN:", train_index, "TEST:", test_index)
    train_and_evaluate_model(model, previsores[train_index], classes[train_index],
                           previsores[test_index], classes[test_index])

W0822 11:39:34.055645  5300 deprecation.py:506] From c:\users\gabriel\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0822 11:39:34.060603  5300 deprecation.py:506] From c:\users\gabriel\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\ops\init_ops.py:97: calling GlorotUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0822 11:39:34.061599  5300 deprecation.py:506] From c:\users\gabriel\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\ops\init_ops.py

TRAIN: [   1    3    4 ... 1996 1997 1998] TEST: [   0    2    6   14   23   24   27   28   30   32   33   36   40   41
   42   43   65   66   71   73   82   84  106  112  113  114  116  117
  118  124  125  127  130  131  132  133  134  139  147  154  168  173
  185  186  191  193  196  202  205  210  213  222  232  233  239  240
  251  256  264  275  279  284  288  290  291  293  295  300  305  308
  312  314  319  328  329  333  338  372  376  382  385  388  397  401
  402  403  406  408  409  415  422  423  427  429  431  437  443  450
  453  456  463  467  468  474  477  487  490  492  497  502  506  509
  511  512  513  519  524  525  526  529  531  533  535  546  549  551
  555  557  560  561  573  576  584  588  590  592  594  603  605  606
  622  623  625  628  631  636  639  641  642  643  651  668  671  672
  673  686  688  690  692  697  698  700  711  729  731  734  738  740
  754  766  769  773  783  787  794  801  806  811  813  817  828  831
  834  840  843  852  873  8

1600/1600 [==============================] - 10s 6ms/sample - loss: 0.1709 - acc: 0.9275
Epoch 62/100
1600/1600 [==============================] - 10s 6ms/sample - loss: 0.1684 - acc: 0.9288
Epoch 63/100
1600/1600 [==============================] - 10s 6ms/sample - loss: 0.1674 - acc: 0.9294
Epoch 64/100
1600/1600 [==============================] - 10s 6ms/sample - loss: 0.1661 - acc: 0.9299
Epoch 65/100
1600/1600 [==============================] - 10s 6ms/sample - loss: 0.1650 - acc: 0.9304
Epoch 66/100
1600/1600 [==============================] - 10s 6ms/sample - loss: 0.1633 - acc: 0.9311
Epoch 67/100
1600/1600 [==============================] - 10s 6ms/sample - loss: 0.1623 - acc: 0.9316
Epoch 68/100
1600/1600 [==============================] - 10s 6ms/sample - loss: 0.1625 - acc: 0.9316
Epoch 69/100
1600/1600 [==============================] - 10s 6ms/sample - loss: 0.1596 - acc: 0.9327
Epoch 70/100
1600/1600 [==============================] - 10s 6ms/sample - loss: 0.1585 - acc: 

1600/1600 [==============================] - 10s 6ms/sample - loss: 0.2808 - acc: 0.8756
Epoch 16/100
1600/1600 [==============================] - 10s 6ms/sample - loss: 0.2795 - acc: 0.8761
Epoch 17/100
1600/1600 [==============================] - 10s 6ms/sample - loss: 0.2778 - acc: 0.8773
Epoch 18/100
1600/1600 [==============================] - 10s 6ms/sample - loss: 0.2750 - acc: 0.8786
Epoch 19/100
1600/1600 [==============================] - 10s 6ms/sample - loss: 0.2725 - acc: 0.8800
Epoch 20/100
1600/1600 [==============================] - 10s 6ms/sample - loss: 0.2697 - acc: 0.8815
Epoch 21/100
1600/1600 [==============================] - 10s 6ms/sample - loss: 0.2691 - acc: 0.8819
Epoch 22/100
1600/1600 [==============================] - 10s 6ms/sample - loss: 0.2639 - acc: 0.8847
Epoch 23/100
1600/1600 [==============================] - 10s 6ms/sample - loss: 0.2571 - acc: 0.8879
Epoch 24/100
1600/1600 [==============================] - 10s 6ms/sample - loss: 0.2526 - acc: 

1600/1600 [==============================] - 10s 6ms/sample - loss: 0.1402 - acc: 0.9410
Epoch 96/100
1600/1600 [==============================] - 10s 6ms/sample - loss: 0.1379 - acc: 0.9420
Epoch 97/100
1600/1600 [==============================] - 10s 6ms/sample - loss: 0.1377 - acc: 0.9420
Epoch 98/100
1600/1600 [==============================] - 10s 6ms/sample - loss: 0.1375 - acc: 0.9422
Epoch 99/100
1600/1600 [==============================] - 10s 6ms/sample - loss: 0.1364 - acc: 0.9426
Epoch 100/100
1600/1600 [==============================] - 10s 6ms/sample - loss: 0.1364 - acc: 0.9423
              precision    recall  f1-score   support

           0       0.72      0.70      0.71     42228
           1       0.69      0.63      0.66     22358
           2       0.74      0.80      0.77     38418

    accuracy                           0.72    103004
   macro avg       0.72      0.71      0.71    103004
weighted avg       0.72      0.72      0.72    103004

Acurácia
0.70960510

1600/1600 [==============================] - 10s 6ms/sample - loss: 0.1952 - acc: 0.9168
Epoch 50/100
1600/1600 [==============================] - 10s 6ms/sample - loss: 0.1937 - acc: 0.9174
Epoch 51/100
1600/1600 [==============================] - 10s 6ms/sample - loss: 0.1911 - acc: 0.9184
Epoch 52/100
1600/1600 [==============================] - 10s 6ms/sample - loss: 0.1905 - acc: 0.9187
Epoch 53/100
1600/1600 [==============================] - 10s 6ms/sample - loss: 0.1879 - acc: 0.9200
Epoch 54/100
1600/1600 [==============================] - 10s 6ms/sample - loss: 0.1862 - acc: 0.9205
Epoch 55/100
1600/1600 [==============================] - 10s 6ms/sample - loss: 0.1835 - acc: 0.9220
Epoch 56/100
1600/1600 [==============================] - 10s 6ms/sample - loss: 0.1822 - acc: 0.9224
Epoch 57/100
1600/1600 [==============================] - 10s 6ms/sample - loss: 0.1826 - acc: 0.9222
Epoch 58/100
1600/1600 [==============================] - 10s 6ms/sample - loss: 0.1797 - acc: 

1600/1600 [==============================] - 10s 6ms/sample - loss: 0.3077 - acc: 0.8620
Epoch 4/100
1600/1600 [==============================] - 10s 6ms/sample - loss: 0.3055 - acc: 0.8627
Epoch 5/100
1600/1600 [==============================] - 10s 6ms/sample - loss: 0.3026 - acc: 0.8641
Epoch 6/100
1600/1600 [==============================] - 10s 6ms/sample - loss: 0.2999 - acc: 0.8655
Epoch 7/100
1600/1600 [==============================] - 10s 6ms/sample - loss: 0.2974 - acc: 0.8667
Epoch 8/100
1600/1600 [==============================] - 10s 6ms/sample - loss: 0.2952 - acc: 0.8682
Epoch 9/100
1600/1600 [==============================] - 10s 6ms/sample - loss: 0.2923 - acc: 0.8691
Epoch 10/100
1600/1600 [==============================] - 10s 6ms/sample - loss: 0.2900 - acc: 0.8705
Epoch 11/100
1600/1600 [==============================] - 10s 6ms/sample - loss: 0.2878 - acc: 0.8713
Epoch 12/100
1600/1600 [==============================] - 10s 6ms/sample - loss: 0.2883 - acc: 0.8704

1600/1600 [==============================] - 10s 6ms/sample - loss: 0.1510 - acc: 0.9350
Epoch 84/100
1600/1600 [==============================] - 10s 6ms/sample - loss: 0.1498 - acc: 0.9358
Epoch 85/100
1600/1600 [==============================] - 10s 6ms/sample - loss: 0.1491 - acc: 0.9360
Epoch 86/100
1600/1600 [==============================] - 10s 6ms/sample - loss: 0.1480 - acc: 0.9364
Epoch 87/100
1600/1600 [==============================] - 10s 6ms/sample - loss: 0.1478 - acc: 0.9371
Epoch 88/100
1600/1600 [==============================] - 10s 7ms/sample - loss: 0.1465 - acc: 0.9375
Epoch 89/100
1600/1600 [==============================] - 10s 6ms/sample - loss: 0.1458 - acc: 0.9374
Epoch 90/100
1600/1600 [==============================] - 10s 6ms/sample - loss: 0.1456 - acc: 0.9373
Epoch 91/100
1600/1600 [==============================] - 10s 6ms/sample - loss: 0.1441 - acc: 0.9384
Epoch 92/100
1600/1600 [==============================] - 10s 6ms/sample - loss: 0.1432 - acc: 

1600/1600 [==============================] - 10s 7ms/sample - loss: 0.2187 - acc: 0.9062
Epoch 38/100
1600/1600 [==============================] - 10s 7ms/sample - loss: 0.2172 - acc: 0.9068
Epoch 39/100
1600/1600 [==============================] - 11s 7ms/sample - loss: 0.2151 - acc: 0.9077
Epoch 40/100
1600/1600 [==============================] - 11s 7ms/sample - loss: 0.2117 - acc: 0.9091
Epoch 41/100
1600/1600 [==============================] - 10s 7ms/sample - loss: 0.2094 - acc: 0.9104
Epoch 42/100
1600/1600 [==============================] - 10s 7ms/sample - loss: 0.2076 - acc: 0.9112
Epoch 43/100
1600/1600 [==============================] - 11s 7ms/sample - loss: 0.2054 - acc: 0.9120
Epoch 44/100
1600/1600 [==============================] - 10s 7ms/sample - loss: 0.2035 - acc: 0.9130
Epoch 45/100
1600/1600 [==============================] - 10s 7ms/sample - loss: 0.2021 - acc: 0.9136
Epoch 46/100
1600/1600 [==============================] - 10s 7ms/sample - loss: 0.1995 - acc: 

In [8]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_16 (Bidirectio (None, 700, 200)          73200     
_________________________________________________________________
dropout_16 (Dropout)         (None, 700, 200)          0         
_________________________________________________________________
bidirectional_17 (Bidirectio (None, 700, 200)          181200    
_________________________________________________________________
dropout_17 (Dropout)         (None, 700, 200)          0         
_________________________________________________________________
bidirectional_18 (Bidirectio (None, 700, 200)          181200    
_________________________________________________________________
dropout_18 (Dropout)         (None, 700, 200)          0         
_________________________________________________________________
bidirectional_19 (Bidirectio (None, 700, 200)         

In [9]:
print('Acurácias total')
print(accu)
accu = np.array(accu)
print(accu.mean())
print('Precision total')
print(precisions)
precisions = np.array(precisions)
print(precisions.mean())
print('Recalls total')
print(recalls)
recalls = np.array(recalls)
print(recalls.mean())
print('F1 total')
print(f1)
f1 = np.array(f1)
print(f1.mean())

Acurácias total
[0.7034039533945324, 0.7096051051888246, 0.7109929760742442, 0.7145767650989309, 0.7115171083927231]
0.710019181629851
Precision total
[0.7197155139788294, 0.7209530564007298, 0.7236323439927882, 0.7293643417221605, 0.723667854300988]
0.7234666220790992
Recalls total
[0.7194756107630065, 0.7220981709448177, 0.7245264294679081, 0.72939383744632, 0.7222577934291088]
0.7235503684102322
F1 total
[0.7185074785763638, 0.7206524189505312, 0.7236837474882478, 0.7284291972852662, 0.7222725033833455]
0.7227090691367509
